# Causal Attention - Behind the Scenes

#### Step 1. Import the tools we'll use

In [2]:
import torch
import torch.nn.functional as F  # For softmax

#### Step 2. Fix the randomness so the numbers stay the same every time

In [3]:
torch.manual_seed(123)

#### Step 3. Create a pretend sentence with 6 words, each word has 3 features
Each word is stored as 3 numbers (kind of like how the model sees words as math)
Imagine:
- Row 1 = the word “The”
- Row 2 = the word “cat”
- Row 3 = the word “ran” … and so on

In [11]:
inputs = torch.rand(6, 3)  # Shape: [6 words, 3 features per word]

#### Step 4. Set up 3 layers to help each word:
Create 3 magical layers that help each word do three things:
- Ask a question (Query)
- Compare with others (Key)
- Share its information (Value)

In [13]:
W_Q = torch.nn.Linear(3, 3, bias=False) # This helps the word ask “What am I looking for?”
W_K = torch.nn.Linear(3, 3, bias=False) # This helps others answer “Here’s what I offer.”
W_V = torch.nn.Linear(3, 3, bias=False) # This shares each word’s actual content

#### Step 5. Generate Query, Key, and Value matrices
Use those layers to turn the 6 words into Query, Key, and Value versions of themselves

In [16]:
Q = W_Q(inputs) # What each word is asking about
K = W_K(inputs) # What each word offers
V = W_V(inputs) # What each word carries

#### Step 6. Compare how much attention each word gives to every other word
Each word now looks at all others and scores how closely they match
- Think of this like a "who should I listen to?" score
- We divide by square root of 3 just to keep scores balanced
###### The result is a 6x6 table:
- Row 1 = how much word 1 listens to others
- Row 2 = how much word 2 listens to others, and so on...

In [46]:
attn_scores = torch.matmul(Q, K.T) / (K.shape[-1] ** 0.5)
print (attn_scores)

tensor([[0.0640, 0.0679, 0.0194, 0.0903, 0.0184, 0.0417],
        [0.0842, 0.1372, 0.0323, 0.1545, 0.0290, 0.0893],
        [0.0259, 0.0333, 0.0087, 0.0410, 0.0079, 0.0209],
        [0.0819, 0.1269, 0.0306, 0.1449, 0.0283, 0.0831],
        [0.0528, 0.0541, 0.0156, 0.0738, 0.0138, 0.0315],
        [0.0950, 0.1266, 0.0323, 0.1542, 0.0284, 0.0785]],
       grad_fn=<DivBackward0>)


#### Step 7. Build the mask to block 'future words'
Now comes the important part: the “no cheating” rule!
- We don’t want the model to look ahead in the sentence.
- So we build a “lower triangular mask” , just a fancy term for:

###### A 6x6 grid like this:
##### [1 0 0 0 0 0]
##### [1 1 0 0 0 0]
##### [1 1 1 0 0 0]
##### [1 1 1 1 0 0]
##### [1 1 1 1 1 0]
##### [1 1 1 1 1 1]

##### This tells each word: “You can only look at yourself and words before you.”


In [23]:
mask = torch.tril(torch.ones(attn_scores.shape[0], attn_scores.shape[1]))

#### Step 8. Apply the mask so attention can't go to future tokens
- We’ll block all the 0s by replacing their scores with -infinity
- This way, when we do softmax next, the model will completely ignore those blocked words

In [28]:
masked_scores = attn_scores.masked_fill(mask == 0, float('-inf'))

#### Step 9. Convert the masked scores into attention weights (probabilities)
- Use softmax to turn the scores into probabilities (like “how much attention should I give?”)
- Now each row adds up to 1, it’s like slicing a pie and giving bigger slices to more important words

In [30]:
attn_weights = F.softmax(masked_scores, dim=-1)

#### Step 10. Combine values using the attention weights
- Multiply the attention weights with the Value matrix
- This gives us the final answer: each word has now combined information from the words it listened to

In [32]:
output = torch.matmul(attn_weights, V)

#### Step 11. Show the final output

In [35]:
print("Final context-aware word embeddings:\n", output)

Final context-aware word embeddings:
 tensor([[-0.3325, -0.1223,  0.2555],
        [-0.5215, -0.1879,  0.1063],
        [-0.3994, -0.1458,  0.0869],
        [-0.4794, -0.1667,  0.0904],
        [-0.4201, -0.1554,  0.0910],
        [-0.4472, -0.1731,  0.0766]], grad_fn=<MmBackward0>)


### This is how models like GPT generate text one word at a time!